In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random
import torch
from torch import nn
import torch.nn.functional as F
import os
import datetime


import import_ipynb
from dqn_p4 import replay_memory
from dqn_p4 import DQN


## simple

In [ ]:
class DQN_search():
    def __init__(s, dqn_model = 0, policy_dqn = 0):
        if (policy_dqn == 0):
            s.policy_dqn = plastic_nn(optimizer="Adam")
        else:
            s.policy_dqn = policy_dqn

        if (dqn_model == 0):
            s.dqn_model = DQN()
        else:
            s.dqn_model = dqn_model
        pass

    def set_DQN(s, dqn_model):
        s.dqn_model = dqn_model
        
    def set_NN(s, policy_dqn):
        s.policy_dqn = policy_dqn

    def set_a_type_array(s, a_type):
        s.atype = a_type

    def set_lr(s, lr):
        s.lr = lr

    def set_nn_topology(s, layers_net):
        s.policy_dqn.delete_layers()
        s.policy_dqn.append_layers(layers_net)
        s.policy_dqn.optimizer_reset()
                
    

    def create_net(s, shape):
        layers_net = []
        layers_net.append(input_layer(shape[0]))
        for i in range(1, len(shape)):
            new_layer = layer(lr = s.lr, prev_size = shape[i-1], my_size=shape[i], activation_type=s.atype[i])
            # new_layer.print_info()
            layers_net.append(new_layer)
        return layers_net

    def train(s, epochs):       
        s.dqn_model.train(s.policy_dqn, epochs, False)

    def update_info_tag(s, m, shape):

        shape_tag= ''.join(str(x) for x in shape)
        s.dqn_model.set_tag(tag = shape_tag+'_'+str(m))
        s.dqn_model.save_info(f'lr: {s.lr} \nshape:{shape} \na1:{s.atype}\n')

    def test(s, n_tests_for_model):
        path = s.dqn_model.path

        s.policy_dqn.load(f"{path}/mc_policy_best")

        return s.dqn_model.test(s.policy_dqn, n_tests_for_model, render = False)
        
                
                
        
    def search(s, n_models, TSR, start_shape, shape_steps, final_shape,
              epochs = 2000, n_tests_for_model = 100,
               adaptive_n_models = False, min_best_result = 0
              ):
        
        shape = start_shape

        steps_dif = shape_steps.shape[0]
        step_i = 0

        flag = False
        adapt_cnt = 0
        while not (np.array_equal(shape, final_shape)):  
            m = 0
            best_result = min_best_result
            while (m < n_models): 
                print(m, shape)
                s.set_nn_topology(s.create_net(shape))
                if (flag):
                    numb = str(m) +'_'+str(adapt_cnt)
                else:
                    numb = m
                s.update_info_tag(numb, shape)

                #train
                s.train(epochs)

                # test model
                test_res, medium_reward, reward_list = s.test(n_tests_for_model)
                
                print('test_res ', test_res)
                print('medium_reward ', medium_reward)

                if (test_res >= TSR):
                    print('success shape: ', shape)
                    return test_res, shape, s.policy_dqn

                if (adaptive_n_models):
                    if (test_res > best_result):
                        m -= 1
                        if (flag == True):
                            adapt_cnt+=1
                        flag = True
                        best_result = test_res
                    else:
                        flag = False
                        adapt_cnt = 0
                m+=1
            
            #shape step
            step_mask = shape_steps[step_i]
            step_i += 1
            step_i %= steps_dif
    
            shape+=step_mask

    
        print('search falied') 
        return False, 0, 0

## comp